# Intrinsic Single-Image HDR Reconstruction

This Colab Notebook helps you testing the HDR reconstruction method 
introduced in our ECCV 2024 paper "Intrinsic Single-Image HDR Reconstruction".

Instructions:

1. Ensure using a GPU by setting "Runtime/change runtime type" to GPU.
2. Install the repository.
3. Read the example image or **put your own LDR images into './images'**. 
4. Convert the image to linear RGB.
5. Run our HDR reconstruction pipeline.
6. [optional] Download the result as EXR file.

In [ ]:
# set tensorflow version to 2.10 for linearization baseline
!pip install --upgrade "tensorflow<=2.10"

In [ ]:
# Clone the repository.
!git clone https://github.com/compphoto/IntrinsicHDR
%cd IntrinsicHDR


In [ ]:
# Install the dependencies.
!pip install pytorch_lightning kornia

In [ ]:
# imports
import tensorflow.compat.v1 as tf
import torch
from matplotlib import pyplot as plt
from IntrinsicHDR.intrinsic_decomposition.common.model_util import load_models
from IntrinsicHDR.dequantize_and_linearize import build_session, build_graph, dequantize_and_linearize
from IntrinsicHDR.inference import load_reconstruction_models, intrinsic_hdr
from IntrinsicHDR.src.utils import read_ldr_image, tonemap

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# load the image to run through the pipeline
img = read_ldr_image('./images/input/sunset_forest.jpg')

First, convert the image to linear RGB:

In [ ]:
# download pretrained weights
!wget https://github.com/compphoto/IntrinsicHDR/releases/download/v1.0/model.ckpt.data-00000-of-00001
!wget https://github.com/compphoto/IntrinsicHDR/releases/download/v1.0/model.ckpt.index 
!wget https://github.com/compphoto/IntrinsicHDR/releases/download/v1.0/model.ckpt.meta

!mkdir -p ./baselines/SingleHDR/checkpoints

!mv model.ckpt.* ./baselines/SingleHDR/checkpoints/.


In [ ]:
# build Tensorflow graph
ldr = tf.placeholder(tf.float32, [None, None, None, 3])
is_training = tf.placeholder(tf.bool)
graph  = build_graph(ldr,is_training)

# root directory
root = '.'

# run the session
with build_session(root) as sess:
    img_lin = dequantize_and_linearize(img,sess,graph,ldr,is_training)

Now, reconstruct the HDR image from the linearized input:

In [ ]:
# load the intrinsic decomposition models
decomp_models = load_models(
    ord_path='vivid_bird_318_300.pt',
    mrg_path='fluent_eon_138_200.pt',
    device = DEVICE
)

weights_url = 'https://github.com/compphoto/IntrinsicHDR/releases/download/v1.0/'

# load the reconstruction models
reconstruction_models = load_reconstruction_models(DEVICE, model_root = weights_url)

# run the pipeline
reconstructed_results = intrinsic_hdr(decomp_models, reconstruction_models, img_lin)

# get the result
hdr = reconstructed_results['rgb_hdr']

Apply Reinhard's tone mapper to display the image:

In [ ]:
import numpy as np
hdr_tm = tonemap(hdr*0.05/np.median(hdr)) # scale the HDR image to a reasonable range
plt.figure(figsize=(16,9))
plt.imshow(hdr_tm)
plt.axis('off')